# Spring's constant measuring

## Importing libraries

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import os
from functools import reduce

from modules import LinearRegression

## Load data

The following code creates a table (dict) with entries that have:
- as key, the name of the file from which the data is read
- as value, the DataFrame returned by pandas when reading that file

Getting data from this table involves using the file from which the data is read as key and then parsing the columns of the corresponding DataFrame.

e.g.:

```
masses_measurements = datasets['misure_masse']
used_objects = masses_measurements[masses_measurements.columns[0]]
masses = masses_measurements[masses_measurements.columns[1]]
```

In [89]:
# collect filenames of exported data

data_dir = "./data"

csv_data = []
excel_data = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        current_file_path = os.path.join(root, file).replace(
            "\\", "/"
        )  # fix the unbearably frustrating flaws of the wanna-be OS... "Windows"
        if "xlsx" in file:
            excel_data.append(current_file_path)
        elif "csv" in file:
            csv_data.append(current_file_path)
    break  # stop at first recursion level: only ./data

print(f"CSV:\n{csv_data}\nEXCEL:\n{excel_data}")

CSV:
['./data/sonar_non-pretensionata_statico.csv', './data/sonar_non-pretensionata_dinamico.csv', './data/sonar_pretensionata_dinamico.csv', './data/sonar_pretensionata_statico.csv']
EXCEL:
['./data/misure_masse.xlsx', './data/calibro_non-pretensionata_statico.xlsx', './data/masse_sonar_non-pretensionata_statico.xlsx', './data/calibro_pretensionata_statico.xlsx', './data/masse_sonar_non-pretensionata_dinamico.xlsx', './data/masse_sonar_pretensionata_dinamico.xlsx', './data/masse_sonar_pretensionata_statico.xlsx']


In [90]:
def read_csv(filename: str):
    data = pd.read_csv(filename, sep=";").replace(",", ".", regex=True)

    data.dropna(inplace=True)
    data.drop(
        index=data.index[0], axis=0, inplace=True
    )  # instrument error causes first value to be nonsensical

    for col in data.columns:
        data[col] = data[col].apply(pd.to_numeric)

    return data


def read_excel(filename: str):
    data = pd.read_excel(filename)
    return data


datasets = dict()

for file in csv_data + excel_data:
    key = os.path.basename(file).split(".")[0]
    if "csv" in file:
        datasets[key] = read_csv(file)
    elif "xlsx" in file:
        datasets[key] = read_excel(file)

print(datasets.keys())

for k in datasets.keys():
    print(f"\nkey: {k}")
    datasets[k].info()

dict_keys(['sonar_non-pretensionata_statico', 'sonar_non-pretensionata_dinamico', 'sonar_pretensionata_dinamico', 'sonar_pretensionata_statico', 'misure_masse', 'calibro_non-pretensionata_statico', 'masse_sonar_non-pretensionata_statico', 'calibro_pretensionata_statico', 'masse_sonar_non-pretensionata_dinamico', 'masse_sonar_pretensionata_dinamico', 'masse_sonar_pretensionata_statico'])

key: sonar_non-pretensionata_statico
<class 'pandas.core.frame.DataFrame'>
Index: 1027 entries, 1 to 1027
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Position (mm) Run #1  1027 non-null   float64
 1   Position (mm) Run #2  1027 non-null   float64
 2   Position (mm) Run #3  1027 non-null   float64
 3   Position (mm) Run #4  1027 non-null   float64
 4   Position (mm) Run #5  1027 non-null   float64
 5   Position (mm) Run #6  1027 non-null   float64
dtypes: float64(6)
memory usage: 56.2 KB

key: sonar_non-preten

## Analyze data

For the **static method**, the sonar's signal is hypothesized to have a very small amplitude, so the measure of the mass' displacement is the mean of the recorded positions.<BR><BR>
For the **dynamic method**, we isolate the peaks of the oscillating signal and compute the mean of the period between two adjacent peaks. From there, the resulting period is used to approximate the frequency of the oscillation.

In [91]:
# sampling frequencies in Hz

static_sampling_frequency = 100
dynamic_sampling_frequency = 200

In [92]:
static_keys = list(filter(lambda k: "statico" in k, datasets.keys()))
dynamic_keys = list(filter(lambda k: "dinamico" in k, datasets.keys()))

print(f"{static_keys}\n{dynamic_keys}")

static_datasets = {k: datasets[k] for k in static_keys}
dynamic_datasets = {k: datasets[k] for k in dynamic_keys}

['sonar_non-pretensionata_statico', 'sonar_pretensionata_statico', 'calibro_non-pretensionata_statico', 'masse_sonar_non-pretensionata_statico', 'calibro_pretensionata_statico', 'masse_sonar_pretensionata_statico']
['sonar_non-pretensionata_dinamico', 'sonar_pretensionata_dinamico', 'masse_sonar_non-pretensionata_dinamico', 'masse_sonar_pretensionata_dinamico']


In [93]:
object_masses = dict()
ds = datasets["misure_masse"]
objects = [str(e) for e in ds.iloc[:,0].array.tolist()]
masses = ds.iloc[:,1].array.tolist()

for i in range(len(objects)):
    object_masses[objects[i]] = masses[i]
print(object_masses)

del objects
del masses
del ds


# inc_masses = np.zeros(10)
# inc_masses[0] = object_masses["massa [g]"][2] + object_masses["massa [g]"][3]
# for i in range(0, 9):
    # inc_masses[i + 1] = inc_masses[i] + object_masses["massa [g]"][i + 4]


def calculate_total_mass(used_masses: str, spring_used: str, has_support_structure: bool = True):
    total_mass = object_masses[spring_used]
    if has_support_structure:
        total_mass += object_masses['piatto'] + object_masses['supporto']

    if not isinstance(used_masses, str):
        return total_mass

    for m in [s for s in used_masses.split(',') if s]:
        if m in object_masses.keys():
            total_mass += object_masses[m]
        else:
            continue

    return total_mass

print(f"testing with 1,2,4,5 (and support structure): {calculate_total_mass('1,,2,4,,5', 'molla pretensionata')}") # it works, noice

{'molla pretensionata': 24.98, 'molla non pretensionata': 15.46, 'piatto': 8.28, 'supporto': 19.87, '1': 19.54, '2': 19.63, '3': 19.86, '4': 20.13, '5': 19.99, '6': 19.66, '7': 19.91, '8': 19.99, '9': 20.01}
testing with 1,2,4,5 (and support structure): 132.42


### Static method

In [102]:
def analyze_calibre_data(dataset_name: str):
    used_spring = 'molla ' + list(filter(lambda e: 'pretensionata' in e, dataset_name.split('_')))[0].replace('-', ' ') # ugly as hell solution
    used_masses = static_datasets[dataset_name].iloc[:, 0].array.tolist()
    mm_lengths = static_datasets[dataset_name].iloc[:, 1].array.tolist()
    in_lengths = static_datasets[dataset_name].iloc[:, 2].array.tolist()
    print(f"\nCALIBRE dataset: {dataset_name}\nspring: {used_spring}\nmasses: {used_masses}\nmm: {mm_lengths}\nin: {in_lengths}")

    results = np.ndarray((len(used_masses), 3))

    for i in range(len(used_masses)):
        results[i] = [calculate_total_mass(used_masses[i], used_spring), mm_lengths[i], in_lengths[i]]
    
    print(results, end='\n------------\n')
    return results



def analyze_sonar_data(dataset_name: str):
    runs = static_datasets[dataset_name]
    used_masses = static_datasets["masse_" + dataset_name].iloc[:, 0].array.tolist()
    used_spring = 'molla ' + list(filter(lambda e: 'pretensionata' in e, dataset_name.split('_')))[0].replace('-', ' ') # ugly as hell solution
    print(f"\nSONAR dataset: {dataset_name}\nmasses: {used_masses}\nspring: {used_spring}")

    results = np.ndarray((len(runs.columns),2))

    i = 0
    for run_name in runs.columns:
        # print(runs[run_name].array)
        signal = runs[run_name]
        results[i] = [calculate_total_mass(used_masses[i], used_spring), np.mean(signal)]
        i += 1
    
    print(results, end='\n------------\n')
    return results



# TODO: store analyzed results and calculate linear regression

def analyze_dataset(dataset_name: str):
    if 'calibro' in dataset_name:
        analyze_calibre_data(dataset_name)
    elif 'sonar' in dataset_name:
        analyze_sonar_data(dataset_name)


for key in filter(lambda e: not 'masse' in e, static_datasets.keys()):
    analyze_dataset(key)


SONAR dataset: sonar_non-pretensionata_statico
masses: [nan, '1,', '1,2', '1,2,3', '1,2,3,4', '1,2,3,4,5']
spring: molla non pretensionata
[[ 4.36100000e+01 -4.50340798e-02]
 [ 6.31500000e+01 -6.20304771e+01]
 [ 8.27800000e+01 -1.23295346e+02]
 [ 1.02640000e+02 -1.85961646e+02]
 [ 1.22770000e+02 -2.48528403e+02]
 [ 1.42760000e+02 -3.11495940e+02]]
------------

SONAR dataset: sonar_pretensionata_statico
masses: ['1,2,3', '1,2,3,4', '1,2,3,4,5', '1,2,3,4,5,6', '1,2,3,4,5,6,7', '1,2,3,4,5,6,7,8', '1,2,3,4,5,6,7,8,9']
spring: molla pretensionata
[[ 1.12160000e+02  1.92896016e-01]
 [ 1.32290000e+02 -1.27056851e+01]
 [ 1.52280000e+02 -2.57116812e+01]
 [ 1.71940000e+02 -3.83418465e+01]
 [ 1.91850000e+02 -5.14526433e+01]
 [ 2.11840000e+02 -6.43117687e+01]
 [ 2.31850000e+02 -7.71967541e+01]]
------------

CALIBRE dataset: calibro_non-pretensionata_statico
spring: molla non pretensionata
masses: ['1,2,3,4,5,6', '1,2,3,4,5', '1,2,3,4']
mm: [88.24, 150.38, 211.24]
in: [3.475, 5.922, 8.314]
[[162

In [95]:
# Analyze static method

averages = np.zeros(7)
for run in range(7):
    averages[run] += np.mean(
        datasets["sonar_pretensionata_statico"][f"Position (mm) Run #{run+1}"]
    )

averages = np.round_(averages, decimals=2)
print(averages)

k = 9.81 * (inc_masses[-1] - inc_masses[3]) / (averages[-1] - averages[0])
print(k)

[  0.19 -12.71 -25.71 -38.34 -51.45 -64.31 -77.2 ]
-0.0


### Dynamic method

In [96]:
# Analyze dynamic method


def discrete_derivative(domain: list, curve: list):
    d = [0]
    for i in range(1, len(curve)):
        d.append((curve[i] - curve[i - 1]) / (domain[i] - domain[i - 1]))
    return d


# windowed peaks isolation
def windowed_peaks(signal: list):
    peaks = []
    for i in range(2, len(signal) - 2, 4):
        if (
            reduce(
                lambda x, y: x * y,
                discrete_derivative(list(range(i - 2, i + 3)), signal[i - 2 : i + 3]),
            )
            < 0
        ):
            peaks.append(max(signal[i - 2 : i + 3]))
    return peaks

In [97]:
def calculate_mean_period(column_id: str):
    wf = data[column_id].array
    for v in wf:
        print(windowed_peaks(wf))


# calculate_mean_period(data[data.columns[0]])

# for col in data.columns:
# calculate_mean_period(col)